# Lesson-03


**Функции из прошлого ДЗ:**

In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [6]:
def hh_parser(vac):
    
    link = 'https://hh.ru/search/vacancy'
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
        }
    
    df = pd.DataFrame(columns =('Название', 'Ссылка', 'Мин. з/п', 'Макс. з/п', 'Валюта', 'Сайт'))
    
    
    for i in range(100):
        params = {
            'st': 'searchVacancy', \
            'text': vac, \
            'search_field': 'name', \
            'search_period': '7', \
            'items_on_page': '50', \
            'page': i
        }

        html = requests.get(link, params=params, headers=headers)

        if html.ok:
            soup = bs(html.text,'lxml')
            vacancy1 = soup.find('div', {'class':'vacancy-serp-item'})
            div_on_page = len(soup.find('div', {'class':"vacancy-serp"}))
            vacancy_site_link = 'https://hh.ru/'

            for i in range(div_on_page):
                try:
                    ### Название вакансии
                    vacancy_name = vacancy1.find('span', {'class':'g-user-content'}).text

                    ### Ссылка на вакансию
                    vacancy_link = vacancy1.find('span', {'class':'g-user-content'})
                    vacancy_link = vacancy_link.next['href']

                    ### З/п на вакансию
                    vacancy_salary = vacancy1.find('div', {'class':'vacancy-serp-item__sidebar'}).next.text
                    vacancy_salary = vacancy_salary.replace('\xa0', '')
                    vacancy_salary = vacancy_salary.replace(' ', '-')
                    vacancy_salary = vacancy_salary.split('-')
                    vacancy_salary_min = vacancy_salary[0]
                    vacancy_salary_max = vacancy_salary[1]
                    vacancy_salary_currency = vacancy_salary[2]


                    try:
                        vacancy_salary_min = int(vacancy_salary_min)
                    except:
                        vacancy_salary_min = float('Nan')

                    try:
                        vacancy_salary_max = int(vacancy_salary_max)
                    except:
                        vacancy_salary_max = float('Nan')

                    if vacancy_salary_currency == ('руб.'):
                        vacancy_salary_currency = 'RUB'
                    elif vacancy_salary_currency == ('EUR'):
                        vacancy_salary_currency = 'EUR'
                    elif vacancy_salary_currency == ('USD'):
                        vacancy_salary_currency = 'USD'
                    else:
                        vacancy_salary_currency = float('Nan')

                    df = df.append(
                        {
                        'Название': vacancy_name,
                        'Ссылка': vacancy_link,
                        'Мин. з/п': float(vacancy_salary_min),
                        'Макс. з/п': float(vacancy_salary_max),
                        'Валюта': vacancy_salary_currency,
                        'Сайт': vacancy_site_link,
                        },
                        ignore_index=True,
                        )
                    vacancy1 = vacancy1.next_sibling
                except:
                    vacancy1 = vacancy1.next_sibling
        if soup.find('a', {'class':'HH-Pager-Controls-Next'}) == None:
            break
    return df

In [7]:
def superjob_parser(vac):
    link = 'https://russia.superjob.ru'
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
        }
    df = pd.DataFrame(columns =('Название', 'Ссылка', 'Мин. з/п', 'Макс. з/п', 'Валюта', 'Сайт'))

    final_link = link + '/vacancy/search/'

    for k in range(30):
            params = {
                'keywords': vac, \
                'page': k, \
                }

            html = requests.get(final_link, params=params, headers=headers)

            if html.ok:
                soup = bs(html.text,'lxml')
                vacancy1 = soup.find('div', {'class':'f-test-vacancy-item'}).parent.parent
                vacancy_on_page = vacancy1.findChildren('div', {'class':'iJCa5'})


                for i in vacancy_on_page:
                    vacancy_name = i.find_next('div', {'class':'CuJz5'}).text

                    vacancy_link = i.find_next('div', {'class':'CuJz5'})
                    vacancy_salary = vacancy_link.next_sibling
                    vacancy_link = vacancy_link.next['href']
                    vacancy_link = link + vacancy_link
                    vacancy_link

                    vacancy_salary = vacancy_salary.text
                    vacancy_salary = vacancy_salary.split('\xa0')
                    
                    if vacancy_salary[0] == 'до':
                        vacancy_salary = [float('Nan'), vacancy_salary[1]+vacancy_salary[2], vacancy_salary[3]]
                    elif vacancy_salary[0] == 'от':
                        vacancy_salary = [vacancy_salary[1]+vacancy_salary[2], float('Nan'), vacancy_salary[3]]
                    elif len(vacancy_salary) == 1:
                        vacancy_salary = [float('Nan'), float('Nan'), float('Nan')]
                    elif vacancy_salary[2] == '-':
                        vacancy_salary = [vacancy_salary[0]+vacancy_salary[1], vacancy_salary[3]+vacancy_salary[4], vacancy_salary[5]]
                    elif len(vacancy_salary) == 3:
                        pass
                    else:
                        vacancy_salary = [float('Nan'), float('Nan'), float('Nan')]
                    
                    if vacancy_salary[2] == ('руб.'):
                        vacancy_salary[2] = 'RUB'
                    elif vacancy_salary[2] == ('EUR'):
                        vacancy_salary[2] = 'EUR'
                    elif vacancy_salary[2] == ('USD'):
                        vacancy_salary[2] = 'USD'
                    else:
                        vacancy_salary[2] = float('Nan')
                    
                    df = df.append(
                        {
                        'Название': vacancy_name,
                        'Ссылка': vacancy_link,
                        'Мин. з/п': float(vacancy_salary[0]),
                        'Макс. з/п': float(vacancy_salary[1]),
                        'Валюта': vacancy_salary[2],
                        'Сайт': link,
                        },
                        ignore_index=True,
                        )


            if soup.find(text='Дальше') == None:
                break
    return df

In [8]:
def hh_superjob_parser(keyword):
    keyword = str(keyword)
    df1 = hh_parser(keyword)
    df2 = superjob_parser(keyword)
    df = pd.concat([df1, df2], axis=0)
    return df

In [9]:
hh_superjob_parser('Python')

,Название,Ссылка,Мин. з/п,Макс. з/п,Валюта,Сайт
0,Python Developer,https://spb.hh.ru/vacancy/35095020?query=Python,NaN,2500.0,USD,https://hh.ru/
1,Программист Python,https://spb.hh.ru/vacancy/36615032?query=Python,170000.0,250000.0,RUB,https://hh.ru/
2,Python Backend Developer (Munchen),https://spb.hh.ru/vacancy/36414126?query=Python,4200.0,6250.0,EUR,https://hh.ru/
3,"Back-end Developer (Python, Django)",https://spb.hh.ru/vacancy/35493930?query=Python,NaN,100000.0,RUB,https://hh.ru/
4,Backend python/Django разработчик,https://spb.hh.ru/vacancy/36622564?query=Python,120000.0,200000.0,RUB,https://hh.ru/
...,...,...,...,...,...,...
63,Инженер телематических служб,https://russia.superjob.ru/vakansii/inzhener-t...,NaN,NaN,NaN,https://russia.superjob.ru
64,Ведущий инженер в отдел серверных систем Linux,https://russia.superjob.ru/vakansii/veduschij-...,NaN,NaN,NaN,https://russia.superjob.ru
65,Инженер в отдел баз данных Hadoop и noSQL,https://russia.superjob.ru/vakansii/inzhener-v...,NaN,NaN,NaN,https://russia.superjob.ru
66,Инженер (программирование),https://russia.superjob.ru/vakansii/inzhener-3...,40000.0,NaN,RUB,https://russia.superjob.ru


**Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД**

In [10]:
from pymongo import MongoClient
from pprint import pprint

In [11]:
client = MongoClient('localhost', 27017)
db = client['Vacancies']

In [12]:
df = hh_superjob_parser('Python')
df.rename(columns={'Название':'name', 'Ссылка':'link', 'Мин. з/п':'minsalary', 'Макс. з/п':'maxsalary', 'Валюта':'currency', 'Сайт':'website'}, inplace=True)
df

,name,link,minsalary,maxsalary,currency,website
0,Python Developer,https://spb.hh.ru/vacancy/35095020?query=Python,NaN,2500.0,USD,https://hh.ru/
1,Программист Python,https://spb.hh.ru/vacancy/36615032?query=Python,170000.0,250000.0,RUB,https://hh.ru/
2,Python Backend Developer (Munchen),https://spb.hh.ru/vacancy/36414126?query=Python,4200.0,6250.0,EUR,https://hh.ru/
3,"Back-end Developer (Python, Django)",https://spb.hh.ru/vacancy/35493930?query=Python,NaN,100000.0,RUB,https://hh.ru/
4,Backend python/Django разработчик,https://spb.hh.ru/vacancy/36622564?query=Python,120000.0,200000.0,RUB,https://hh.ru/
...,...,...,...,...,...,...
63,Инженер телематических служб,https://russia.superjob.ru/vakansii/inzhener-t...,NaN,NaN,NaN,https://russia.superjob.ru
64,Ведущий инженер в отдел серверных систем Linux,https://russia.superjob.ru/vakansii/veduschij-...,NaN,NaN,NaN,https://russia.superjob.ru
65,Инженер в отдел баз данных Hadoop и noSQL,https://russia.superjob.ru/vakansii/inzhener-v...,NaN,NaN,NaN,https://russia.superjob.ru
66,Инженер (программирование),https://russia.superjob.ru/vakansii/inzhener-3...,40000.0,NaN,RUB,https://russia.superjob.ru


In [13]:
df_dict = df.to_dict('records')
db.vacancies.delete_many({})
db.vacancies.insert_many(df_dict)

**Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.**

In [14]:
i = 0
while i == 0:
    try:
        salary = float(input())
        i=1
    except:
        print('Введите корректное число!')

sdf
Введите корректное число!
230000


In [15]:
objects = db.vacancies.find({'maxsalary': {'$gt': salary} })
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd5'),
 'currency': 'RUB',
 'link': 'https://spb.hh.ru/vacancy/36615032?query=Python',
 'maxsalary': 250000.0,
 'minsalary': 170000.0,
 'name': 'Программист Python',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c5ff6'),
 'currency': 'RUB',
 'link': 'https://spb.hh.ru/vacancy/36101850?query=Python',
 'maxsalary': 250000.0,
 'minsalary': nan,
 'name': 'Разработчик Python (Django)',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c6007'),
 'currency': 'RUB',
 'link': 'https://spb.hh.ru/vacancy/36519460?query=Python',
 'maxsalary': 250000.0,
 'minsalary': 150000.0,
 'name': 'Разработчик Python / Django',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c6024'),
 'currency': 'RUB',
 'link': 'https://spb.hh.ru/vacancy/35768680?query=Python',
 'maxsalary': 350000.0,
 'minsalary': 200000.0,
 'name': 'Разработчик на PYTHON',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c6046')

**Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.**

In [18]:
objects = db.vacancies.find({}).limit(1)
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd4'),
 'currency': 'USD',
 'link': 'https://spb.hh.ru/vacancy/35095020?query=Python',
 'maxsalary': 2500.0,
 'minsalary': nan,
 'name': 'Python Developer',
 'website': 'https://hh.ru/'}


In [19]:
df = hh_superjob_parser('Python')
df.rename(columns={'Название':'name', 'Ссылка':'link', 'Мин. з/п':'minsalary', 'Макс. з/п':'maxsalary', 'Валюта':'currency', 'Сайт':'website'}, inplace=True)
df
df_dict_for_updates = df.to_dict('records')

In [52]:
for i in df_dict_for_updates:
    db.vacancies.update_one({'link': i['link']}, 
                                 {'$set': {'name': i['name'],
                                            'link': i['link'],
                                            'minsalary': i['minsalary'],
                                            'maxsalary': i['maxsalary'],
                                            'currency': i['currency'],
                                            'website': i['website']},},
                                  upsert=True
                                  )

In [53]:
objects = db.vacancies.find( {} ).limit(5)
for obj in objects:
    pprint(obj)

{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd4'),
 'currency': 'USD',
 'link': 'https://spb.hh.ru/vacancy/35095020?query=Python',
 'maxsalary': 2500.0,
 'minsalary': nan,
 'name': 'Python Developer',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd5'),
 'currency': 'RUB',
 'link': 'https://spb.hh.ru/vacancy/36615032?query=Python',
 'maxsalary': 250000.0,
 'minsalary': 170000.0,
 'name': 'Программист Python',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd6'),
 'currency': 'EUR',
 'link': 'https://spb.hh.ru/vacancy/36414126?query=Python',
 'maxsalary': 6250.0,
 'minsalary': 4200.0,
 'name': 'Python Backend Developer (Munchen)',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd7'),
 'currency': 'RUB',
 'link': 'https://spb.hh.ru/vacancy/35493930?query=Python',
 'maxsalary': 100000.0,
 'minsalary': nan,
 'name': 'Back-end Developer (Python, Django)',
 'website': 'https://hh.ru/'}
{'_id': ObjectId('5e98dc7ebb018b55ac2c5fd8'),